In [1]:
from datasets import Dataset

In [2]:
#val_dataset = Dataset.load_from_disk("../oqa_v1.0_shuffled_split/bin/val")
val_dataset = Dataset.load_from_disk("tmp/bin/val")

In [3]:
val_dataset

Dataset({
    features: ['question', 'context', 'answers', 'answer_sentence', 'topic', 'reference', 'id'],
    num_rows: 55
})

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
from models.llm_inference import GpuInference
tt5 = GpuInference(model_checkpoint = "google/t5-efficient-tiny",
             tokenizer_checkpoint = "google/t5-efficient-tiny",
             int8=False,
             dataset=val_dataset,
            prompt_fmt="uniqa")
prompts = tt5.get_prompts()

In [29]:
prompts[0]

{'answer': 'within about the first 100 days',
 'prompt': 'regarding mandibular expansion with the lip bumper, when does the greatest amount of expansion occur?\na critical aspect of nonextraction therapy is the creation of space. the results from this study show that lip bumper therapy is an effective means of accomplishing this task. other studies have demonstrated similar results using lip bumpers but have not focused on the attenuation in expansion that occurs during treatment. a major concern for orthodontists is treatment effectiveness and efficiency. it would be beneficial to know the specific amount of time required to achieve sufficient expansion using lip bumper therapy. this information would benefit the clinician by eliminating unnecessary lip bumper wear. this would obviously be desirable for the patients as well. the expansion achieved during lip bumper therapy is evenly distributed during treatment. however, the results clearly demonstrate that the expansion occurred unev

In [34]:
a = tt5.genseq(prompts)

Parameter 'function'=<function GpuInference.genseq.<locals>.<lambda> at 0x00000171C914EC10> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/55 [00:00<?, ? examples/s]

In [37]:
next(iter(a))

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[1918,  388,   26,  ...,    0,    0,    0],
        [1918,    8,    3,  ...,    0,    0,    0],
        [1918,  197,  102,  ...,    0,    0,    0],
        [1918,  388,   26,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}